In [0]:
!pip3 install locust

In [0]:
import locust
print(locust.__version__)

In [0]:
locust

In [0]:
from locust import HttpUser, task

class HelloWorldUser(HttpUser):
    @task
    def hello_world(self):
        self.client.get("/hello")
        self.client.get("/world")

In [0]:
print("hello")

In [0]:
import dataikuapi

client = dataikuapi.APINodeClient("http://k8s-default-dkumadse-f38d97347b-066209af6eb01332.elb.us-east-1.amazonaws.com:12000", "chat")

result = client.run_function("generate",
        user_query = "What is dqm?")
print("Function result: %s" % result.get("response"))


In [0]:
import dataikuapi

client = dataikuapi.APINodeClient("http://k8s-default-dkumadse-f38d97347b-066209af6eb01332.elb.us-east-1.amazonaws.com:80", "chat")

result = client.run_function("read_multiple_files",
        files = [{"filename":"file1.txt","content":"SGVsbG8gd29ybGQh"},{"filename":"file2.txt","content":"VGhpcyBpcyBhIHRlc3QgZmlsZS4="}])
print("Function result: %s" % result.get("response"))


In [0]:
{'combined_text': '--- file1.txt ---\nHello world!\n\n--- file2.txt ---\nThis is a test file.\n', 'file_count': 2}

In [15]:
import os
import uuid
import logging
import shutil
import dataiku
import base64
import io
from retriever import ModelDefination
from process_and_embed_uploaded_files import process_and_embed_uploaded_files


host = "https://cdl-dku-desi-p.commercial-datalake-prod.awscloud.abbvienet.com/"
apiKey = "dkuaps-9ALuuZLhFJg9dTcrSgPMcsdtfP8bpPXC"
os.environ["DKU_CURRENT_PROJECT_KEY"] = "GENAIPOC" 
dataiku.set_remote_dss(host, apiKey, no_check_certificate=True)

# Other configuration parameters
EMBEDDING_MODEL = "custom:iliad-plugin-conn-prod:text-embedding-ada-002"
LLM_MODEL = "custom:iliad-plugin-conn-prod:gpt-4o"

def read_multiple_files(data): 
    """
    Dataiku API endpoint to read and combine content from multiple uploaded files.

    Args:
        data (dict): Dictionary containing base64 encoded files.
                     Expected format: {"files": [{"filename": ..., "content": ...}, ...]}

    Returns:
        dict: Combined content of all files.
    """
    try:
        uploaded_files = data.get("files", [])

        if not uploaded_files:
            return {"error": "No files received."}

        all_contents = []

        for file_info in uploaded_files:
            filename = file_info.get("filename")
            content_b64 = file_info.get("content")

            if not filename or not content_b64:
                continue

            # Decode base64 content
            file_bytes = base64.b64decode(content_b64)
            file_stream = io.BytesIO(file_bytes)

            # Assuming text-based files (e.g., .txt, .csv, .json)
            try:
                file_text = file_stream.read().decode("utf-8")
            except UnicodeDecodeError:
                file_text = "Unable to decode file: " + filename

            all_contents.append(f"--- {filename} ---\n{file_text}\n")

        model = ModelDefination(embedding_model=EMBEDDING_MODEL, llm_id=LLM_MODEL)

        # Process and write to managed folder
        result = process_and_embed_uploaded_files(all_contents, model)
        # print(result)
        
        return {
            "combined_text": "\n".join(all_contents),
            "file_count": len(all_contents),
            "result": result
        }

    except Exception as e:
        return {
            "error": "Failed to process files.",
            "details": str(e)
        }

In [24]:
import dataiku
import pandas as pd
import re
import pickle
from typing import List, Tuple

def extract_files_from_combined_text(combined_text: str) -> List[Tuple[str, str]]:
    """
    Extract filename and content pairs from the combined_text.
    """
    pattern = r"--- (.*?) ---\n(.*?)(?=(?:---|$))"
    matches = re.findall(pattern, combined_text, re.DOTALL)
    if not matches:
        print("No matches found in combined_text.")
    return [(filename.strip(), content.strip()) for filename, content in matches]


def chunk_text(text: str, chunk_size: int = 500, overlap: int = 100) -> List[str]:
    """
    Simple reusable chunking function.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks


def process_and_embed_uploaded_files(output_dict: dict, model):
    combined_text = output_dict[0]
    print("Debug: Combined Text:\n", combined_text)

    file_entries = extract_files_from_combined_text(combined_text)
    print("Debug: Extracted Files:\n", file_entries)

    all_chunks = []

    for filename, content in file_entries:
        chunks = chunk_text(content, chunk_size=100, overlap=50)
        if not chunks:
            print(f"No chunks generated for {filename}.")
        print(f"Debug: Chunks for {filename}:\n", chunks)

        if chunks:
            embeddings = model.embedding_model.embed_documents(chunks)
#             print(f"Debug: Embeddings for {filename}:\n", embeddings)

            chunk_data = [
                {
                    "chunk_text": chunk,
                    "metadata": {"filename": filename}
                }
                for chunk, embedding in zip(chunks, embeddings)
            ]
            all_chunks.extend(chunk_data)

    new_df = pd.DataFrame(all_chunks)
    print("Debug: New DataFrame:\n", new_df)

    embedding_dataset = dataiku.Dataset("input_data_chunked")
    try:
        existing_df = embedding_dataset.get_dataframe()
    except Exception as e:
        existing_df = pd.DataFrame(columns=["chunk_text", "metadata"])

#     combined_df = pd.concat([existing_df, new_df], ignore_index=True)
#     print("Debug: Combined DataFrame:\n", combined_df)

#     embedding_dataset.write_with_schema(combined_df)

    return {
        "status": "success",
        "chunks_added": len(new_df),
        "total_rows": len(new_df)
    }

In [25]:
dict = {
   "files": [
      {
         "filename": "file1.txt",
         "content": "SGVsbG8sIHRoaXMgaXMgYSBuZXcgbWVzc2FnZSE="
      },
      {
         "filename": "file2.txt",
         "content": "VGhpcyBpcyBhbm90aGVyIHRlc3QgZmlsZS4="
      }
   ]
}

print(read_multiple_files(dict))

2025-05-15 04:39:43,348 - INFO - Initialized LangChain embedding model: custom:iliad-plugin-conn-prod:text-embedding-ada-002
2025-05-15 04:39:43,349 - INFO - Initialized LangChain LLM: custom:iliad-plugin-conn-prod:gpt-4o
2025-05-15 04:39:43,350 - INFO - Model Definition initialized with vector store type: FAISS
2025-05-15 04:39:43,351 - INFO - Performing embedding of 1 texts
/data/dataiku/dss_data/code-envs/python/cdl-p-diku-psbts-py39-llm-env/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdl-dku-desi-p.commercial-datalake-prod.awscloud.abbvienet.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2025-05-15 04:39:43,375 - INFO - Finished a chunk. Embedded 1 of 1 texts
2025-05-15 04:39:43,376 - INFO - Done performing embedding of 1 texts
2025-05-15 04:39:43,378 - INFO - Reading dataset GENAIPOC.inpu

Debug: Combined Text:
 --- file1.txt ---
Hello, this is a new message!

Debug: Extracted Files:
 [('file1.txt', 'Hello, this is a new message!')]
Debug: Chunks for file1.txt:
 ['Hello, this is a new message!']
Debug: New DataFrame:
                       chunk_text                   metadata
0  Hello, this is a new message!  {'filename': 'file1.txt'}
{'combined_text': '--- file1.txt ---\nHello, this is a new message!\n\n--- file2.txt ---\nThis is another test file.\n', 'file_count': 2, 'result': {'status': 'success', 'chunks_added': 1, 'total_rows': 1}}


In [28]:
import os
import dataiku
import base64
import io
import pandas as pd
import re
from retriever import ModelDefination


# DSS connection setup
host = "https://cdl-dku-desi-p.commercial-datalake-prod.awscloud.abbvienet.com/"
apiKey = "dkuaps-9ALuuZLhFJg9dTcrSgPMcsdtfP8bpPXC"
os.environ["DKU_CURRENT_PROJECT_KEY"] = "GENAIPOC"
dataiku.set_remote_dss(host, apiKey, no_check_certificate=True)

# Model configuration
EMBEDDING_MODEL = "custom:iliad-plugin-conn-prod:text-embedding-ada-002"
LLM_MODEL = "custom:iliad-plugin-conn-prod:gpt-4o"


def read_multiple_files(data):
    """
    Dataiku API endpoint to read and combine content from multiple uploaded files.

    Args:
        data (dict): Dictionary containing base64 encoded files.
                     Expected format: {"files": [{"filename": ..., "content": ...}, ...]}

    Returns:
        dict: Combined content of all files.
    """
    try:
        uploaded_files = data.get("files", [])
        if not uploaded_files:
            return {"error": "No files received."}

        all_contents = []

        for file_info in uploaded_files:
            filename = file_info.get("filename")
            content_b64 = file_info.get("content")

            if not filename or not content_b64:
                continue

            # Decode base64 content
            file_bytes = base64.b64decode(content_b64)
            file_stream = io.BytesIO(file_bytes)

            # Assuming text-based files (e.g., .txt, .csv, .json)
            try:
                file_text = file_stream.read().decode("utf-8")
            except UnicodeDecodeError:
                file_text = f"Unable to decode file: {filename}"

            all_contents.append(f"--- {filename} ---\n{file_text}\n")

        # Combine all file contents into a single string
        combined_text = "\n".join(all_contents)
        print("Debug: Combined Text:\n", combined_text)

        # Initialize the model
        model = ModelDefination(embedding_model=EMBEDDING_MODEL, llm_id=LLM_MODEL)

        # Process and embed files
        result = process_and_embed_uploaded_files({"combined_text": combined_text}, model)

        return {
            "combined_text": combined_text,
            "file_count": len(uploaded_files),
            "result": result
        }

    except Exception as e:
        return {
            "error": "Failed to process files.",
            "details": str(e)
        }


def extract_files_from_combined_text(combined_text: str):
    """
    Extract filename and content pairs from the combined_text.
    """
    # Updated regex to correctly match multiple files
    pattern = r"--- (.*?) ---\n(.*?)(?=\n---|$)"
    matches = re.findall(pattern, combined_text, re.DOTALL)
    print("Debug: Extracted Files:\n", matches)
    return [(filename.strip(), content.strip()) for filename, content in matches]


def chunk_text(text: str, chunk_size: int = 500, overlap: int = 100):
    """
    Simple reusable chunking function.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks


def process_and_embed_uploaded_files(output_dict: dict, model):
    """
    Process the uploaded files' text, chunk and embed them, and store back to the same managed folder.
    """
    combined_text = output_dict["combined_text"]
    file_entries = extract_files_from_combined_text(combined_text)

    all_chunks = []

    for filename, content in file_entries:
        # Chunk the text
        chunks = chunk_text(content, chunk_size=500, overlap=100)
        print(f"Debug: Chunks for {filename}:\n", chunks)

        # Embedding logic (uncomment and replace with your actual embedding logic)
        # embeddings = model.embedding_model.embed_documents(chunks)
        embeddings = [None] * len(chunks)  # Placeholder embeddings for debugging

        # Prepare chunk data
        chunk_data = [
            {
                "chunk_text": chunk,
                "metadata": {"filename": filename}
            }
            for chunk in chunks  # zip(chunks, embeddings) if embeddings are available
        ]
        all_chunks.extend(chunk_data)

    # Convert to DataFrame
    new_df = pd.DataFrame(all_chunks)
    print("Debug: New DataFrame:\n", new_df)

    # Load existing DataFrame from managed folder (as pickle)
    embedding_dataset = dataiku.Dataset("input_data_chunked")

    try:
        existing_df = embedding_dataset.get_dataframe()
    except Exception:
        # If the dataset doesn't exist, start with an empty DataFrame
        existing_df = pd.DataFrame(columns=["chunk_text", "metadata"])

    # Combine with new data
    combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    print("Debug: Combined DataFrame:\n", combined_df)

    # Save back to the dataset
    embedding_dataset.write_with_schema(combined_df)

    return {
        "status": "success",
        "chunks_added": len(new_df),
        "total_rows": len(combined_df)
    }




In [29]:
# Example input
dict_data = {
    "files": [
        {
            "filename": "file1.txt",
            "content": "SGVsbG8sIHRoaXMgaXMgYSBuZXcgbWVzc2FnZSE="
        },
        {
            "filename": "file2.txt",
            "content": "VGhpcyBpcyBhbm90aGVyIHRlc3QgZmlsZS4="
        }
    ]
}

print(read_multiple_files(dict_data))

2025-05-15 04:42:52,801 - INFO - Initialized LangChain embedding model: custom:iliad-plugin-conn-prod:text-embedding-ada-002
2025-05-15 04:42:52,802 - INFO - Initialized LangChain LLM: custom:iliad-plugin-conn-prod:gpt-4o
2025-05-15 04:42:52,803 - INFO - Model Definition initialized with vector store type: FAISS
2025-05-15 04:42:52,806 - INFO - Reading dataset GENAIPOC.input_data_chunked as dataframe
/data/dataiku/dss_data/code-envs/python/cdl-p-diku-psbts-py39-llm-env/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdl-dku-desi-p.commercial-datalake-prod.awscloud.abbvienet.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/data/dataiku/dss_data/code-envs/python/cdl-p-diku-psbts-py39-llm-env/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request i

Debug: Combined Text:
 --- file1.txt ---
Hello, this is a new message!

--- file2.txt ---
This is another test file.

Debug: Extracted Files:
 [('file1.txt', 'Hello, this is a new message!\n'), ('file2.txt', 'This is another test file.')]
Debug: Chunks for file1.txt:
 ['Hello, this is a new message!']
Debug: Chunks for file2.txt:
 ['This is another test file.']
Debug: New DataFrame:
                       chunk_text                   metadata
0  Hello, this is a new message!  {'filename': 'file1.txt'}
1     This is another test file.  {'filename': 'file2.txt'}
Debug: Combined DataFrame:
                                              chunk_text                                           metadata                                         embeddings
0     ### Extracted Text:\nPSIT Patient Journey AI M...  {"file_name": "Phase 1 Requirements - PSIT Pat...                                                NaN
1     . The funnel image below illustrates the patie...  {"file_name": "Phase 1 Requireme

In [42]:
import os
import dataiku
import base64
import io
import pandas as pd
import re
from retriever import ModelDefination

# DSS connection setup
host = "https://cdl-dku-desi-p.commercial-datalake-prod.awscloud.abbvienet.com/"
apiKey = "dkuaps-9ALuuZLhFJg9dTcrSgPMcsdtfP8bpPXC"
os.environ["DKU_CURRENT_PROJECT_KEY"] = "GENAIPOC"
dataiku.set_remote_dss(host, apiKey, no_check_certificate=True)

# Model configuration
EMBEDDING_MODEL = "custom:iliad-plugin-conn-prod:text-embedding-ada-002"
LLM_MODEL = "custom:iliad-plugin-conn-prod:gpt-4o"


def read_multiple_files(data):
    """
    Dataiku API endpoint to read and combine content from multiple uploaded files.

    Args:
        data (dict): Dictionary containing base64 encoded files.
                     Expected format: {"files": [{"filename": ..., "content": ...}, ...]}

    Returns:
        dict: Combined content of all files.
    """
    try:
        uploaded_files = data.get("files", [])
        if not uploaded_files:
            return {"error": "No files received."}

        all_contents = []

        for file_info in uploaded_files:
            filename = file_info.get("filename")
            content_b64 = file_info.get("content")

            if not filename or not content_b64:
                continue

            # Decode base64 content
            file_bytes = base64.b64decode(content_b64)
            file_stream = io.BytesIO(file_bytes)

            # Assuming text-based files (e.g., .txt, .csv, .json)
            try:
                file_text = file_stream.read().decode("utf-8")
            except UnicodeDecodeError:
                file_text = f"Unable to decode file: {filename}"

            all_contents.append(f"--- {filename} ---\n{file_text}\n")

        # Combine all file contents into a single string
        combined_text = "\n".join(all_contents)
        print("Debug: Combined Text:\n", combined_text)

        # Initialize the model
        model = ModelDefination(embedding_model=EMBEDDING_MODEL, llm_id=LLM_MODEL)

        # Process and embed files (write to input_data_chunked dataset)
        result = process_and_embed_uploaded_files({"combined_text": combined_text}, model)

        #✅ Trigger the plugin recipe
        client = dataiku.api_client()
        project = client.get_project("GENAIPOC")  # Use your actual project key
        recipe = project.get_recipe("compute_input_data_chunked_embedded")  # Recipe name
        job = recipe.run()
        job.wait_for_completion()
        print("✅ Plugin recipe 'compute_input_data_chunked_embedded' executed.")

        return {
            "combined_text": combined_text,
            "file_count": len(uploaded_files),
            "result": result,
            "plugin_status": "triggered"
        }

    except Exception as e:
        return {
            "error": "Failed to process files.",
            "details": str(e)
        }


def extract_files_from_combined_text(combined_text: str):
    """
    Extract filename and content pairs from the combined_text.
    """
    # Updated regex to correctly match multiple files
    pattern = r"--- (.*?) ---\n(.*?)(?=\n---|$)"
    matches = re.findall(pattern, combined_text, re.DOTALL)
    print("Debug: Extracted Files:\n", matches)
    return [(filename.strip(), content.strip()) for filename, content in matches]


def chunk_text(text: str, chunk_size: int = 500, overlap: int = 100):
    """
    Simple reusable chunking function.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks


def process_and_embed_uploaded_files(output_dict: dict, model):
    """
    Process the uploaded files' text, chunk and embed them, and store back to the same managed folder.
    Ensures:
        - Proper metadata format for each chunk.
        - Duplicate file names are not reprocessed.
    """
    combined_text = output_dict["combined_text"]
    file_entries = extract_files_from_combined_text(combined_text)

    # Load existing DataFrame from managed folder (as pickle)
    embedding_dataset = dataiku.Dataset("input_data_chunked")

    try:
        existing_df = embedding_dataset.get_dataframe()
    except Exception:
        existing_df = pd.DataFrame(columns=["chunk_text", "metadata"])

    # Extract existing file names from metadata
    existing_file_names = set()
    if not existing_df.empty:
        existing_file_names = {
            meta.get("file_name")
            for meta in existing_df["metadata"].dropna().apply(lambda x: eval(x) if isinstance(x, str) else x)
        }

    all_chunks = []

    for filename, content in file_entries:
        if filename in existing_file_names:
            print(f"Skipping {filename} as it already exists in the dataset.")
            continue

        # Chunk the text
        chunks = chunk_text(content, chunk_size=500, overlap=100)
        print(f"Debug: Chunks for {filename}:\n", chunks)

        # Embedding logic (placeholder)
        embeddings = [None] * len(chunks)  # Replace with actual embeddings if needed

        # Prepare chunk data with proper metadata format
        chunk_data = [
            {
                "chunk_text": chunk,
                "metadata": {
                    "file_name": filename,
                    "image_links": "",  # Assuming no image_links for now
                    "chunk_id": f"{filename}_chunk_{i+1}",
                    "chunk_order": i + 1
                }
            }
            for i, chunk in enumerate(chunks)
        ]

        all_chunks.extend(chunk_data)

    # If no new chunks, return
    if not all_chunks:
        return {
            "status": "no_new_chunks",
            "message": "No new data was added because all files already exist.",
            "chunks_added": 0,
            "total_rows": len(existing_df)
        }

    # Convert to DataFrame
    new_df = pd.DataFrame(all_chunks)

    # Combine with existing data
    combined_df = pd.concat([existing_df, new_df], ignore_index=True)

    # Save back to the dataset
    embedding_dataset.write_with_schema(combined_df)

    return {
        "status": "success",
        "chunks_added": len(new_df),
        "total_rows": len(combined_df)
    }




In [43]:
# Example input
dict_data = {
    "files": [
        {
            "filename": "file3.txt",
            "content": "U29tZSBjb250ZW50IGZvciBmaWxlMy4="
        }
#         {
#             "filename": "file4.txt",
#             "content": "V2VsbCwgSG93IGFyZSB5b3U/"
#         }
    ]
}

print(read_multiple_files(dict_data))

2025-05-15 05:32:41,908 - INFO - Initialized LangChain embedding model: custom:iliad-plugin-conn-prod:text-embedding-ada-002
2025-05-15 05:32:41,909 - INFO - Initialized LangChain LLM: custom:iliad-plugin-conn-prod:gpt-4o
2025-05-15 05:32:41,909 - INFO - Model Definition initialized with vector store type: FAISS
2025-05-15 05:32:41,910 - INFO - Reading dataset GENAIPOC.input_data_chunked as dataframe
/data/dataiku/dss_data/code-envs/python/cdl-p-diku-psbts-py39-llm-env/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdl-dku-desi-p.commercial-datalake-prod.awscloud.abbvienet.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/data/dataiku/dss_data/code-envs/python/cdl-p-diku-psbts-py39-llm-env/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request i

Debug: Combined Text:
 --- file3.txt ---
Some content for file3.

Debug: Extracted Files:
 [('file3.txt', 'Some content for file3.')]
Skipping file3.txt as it already exists in the dataset.
{'error': 'Failed to process files.', 'details': "Recipe has unsupported output type COMPUTABLE_RETRIEVABLE_KNOWLEDGE, can't run it"}


/data/dataiku/dss_data/code-envs/python/cdl-p-diku-psbts-py39-llm-env/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdl-dku-desi-p.commercial-datalake-prod.awscloud.abbvienet.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
